# Pipline Project

The goal is to write a pipeline that will push data from the source file, `cars.csv`, and push it through some filters and print out the results.

In [7]:
import csv
from contextlib import contextmanager

file = '/mnt/data-ubuntu/Projects/Learning_PY_hardway/data/deep_dive/cars.csv' 

idx_car = 0
idx_mpg = 1
idx_cylinders = 2
idx_displacement = 3
idx_horsepower = 4
idx_weight = 5
idx_acceleration = 6
idx_model = 7
idx_origin = 8

converters = (str, float, int, float, float, float, float, int, str)

In [21]:
def data_reader(f_name):
    """
    Read data from f_name.
    """
    with open(f_name) as f:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        yield from csv.reader(f, dialect = dialect)

def data_parser(f_name, converters):
    """
    Change data type accordingly.
    """
    data = data_reader(file)
    next(data) # Skip the header.
    for row in data:
        row = [converter(e) for converter, e in zip(converters, row)]
        yield row
        
def coroutine(fn):
    """
    Coroutine decorator.
    """
    def inner(*args, **kwargs):
        g = fn(*args, **kwargs)
        next(g)
        return g
    return inner

@coroutine
def data_filter(fn_filter, next_coroutine):
    """
    Filter data based on fn_filter and send data to next_coroutine.
    """
    while True:
        data = yield
        if fn_filter(data):
            next_coroutine.send(data)

@coroutine
def printer():
    """
    Print the results.
    """
    while True:
        data = yield
        print(data)
    
@coroutine
def pipline(*filters_words):
    """
    The pipline to process the data.
    """
    p = printer()
    
    for filters_word in filters_words:
        p = data_filter(lambda d, v=filters_word: v in d[0], p)
    
    while True:
        received = yield
        p.send(received)

In [22]:
data = data_parser(file, converters)
p = pipline('Toyota', 'Mark')
for row in data:
    p.send(row)

['Toyota Corolla Mark ii', 24.0, 4, 113.0, 95.0, 2372.0, 15.0, 70, 'Japan']
['Toyota Corolla Mark II (sw)', 23.0, 4, 120.0, 97.0, 2506.0, 14.5, 72, 'Japan']
['Toyota Mark II', 20.0, 6, 156.0, 122.0, 2807.0, 13.5, 73, 'Japan']
['Toyota Mark II', 19.0, 6, 156.0, 108.0, 2930.0, 15.5, 76, 'Japan']
